# Two-View Estimator

> Ayush Baid, Frank Dellaert

A notebook to investigate the TwoViewEstimator performance.

In [2]:
import gtsam
import numpy as np
from gtsam import Cal3Bundler, EssentialMatrix, Pose3, Unit3

import gtsfm.utils.geometry_comparisons as comp_utils
import gtsfm.utils.io as io_utils
from gtsfm.two_view_estimator import TwoViewEstimator
from gtsfm.common.keypoints import Keypoints

GTSAM_EXAMPLE_FILE = "5pointExample1.txt"
EXAMPLE_DATA = io_utils.read_bal(gtsam.findExampleDataFile(GTSAM_EXAMPLE_FILE))

In [3]:
# Create estimator
two_view_estimator = TwoViewEstimator(
    matcher=None, verifier=None, inlier_support_processor=None,
    bundle_adjust_2view=True, eval_threshold_px=4
)

In [4]:
two_view_estimator

In [5]:
# Extract example poses.
i1Ri2 = EXAMPLE_DATA.get_camera(1).pose().rotation()
i1ti2 = EXAMPLE_DATA.get_camera(1).pose().translation()
i2Ti1 = Pose3(i1Ri2, i1ti2).inverse()
i2Ei1 = EssentialMatrix(i2Ti1.rotation(), Unit3(i2Ti1.translation()))

In [6]:
# Prepare tracks from the measurements.
num_points = 5
normalized_coordinates_i1 = []
normalized_coordinates_i2 = []
for i in range(num_points):
    track = EXAMPLE_DATA.get_track(i)
    normalized_coordinates_i1.append(track.measurement(0)[1])
    normalized_coordinates_i2.append(track.measurement(1)[1])
normalized_coordinates_i1 = np.array(normalized_coordinates_i1)
normalized_coordinates_i2 = np.array(normalized_coordinates_i2)

In [7]:
# Perform bundle adjustment.
i2Ri1_optimized, i2Ui1_optimized, corr_idxs = two_view_estimator.bundle_adjust(
    keypoints_i1=Keypoints(normalized_coordinates_i1),
    keypoints_i2=Keypoints(normalized_coordinates_i2),
    verified_corr_idxs=np.hstack(
        [np.arange(normalized_coordinates_i1.shape[0]).reshape(-1, 1)] * 2),
    camera_intrinsics_i1=Cal3Bundler(),
    camera_intrinsics_i2=Cal3Bundler(),
    i2Ri1_initial=i2Ei1.rotation(),
    i2Ui1_initial=i2Ei1.direction(),
)

[2022-05-06 14:38:05,602 DEBUG two_view_estimator.py line 172 71509] Performed DA in 0.001238 seconds.
[2022-05-06 14:38:05,603 INFO bundle_adjustment.py line 193 71509] Input: 5 tracks on 2 cameras

[2022-05-06 14:38:05,609 INFO bundle_adjustment.py line 226 71509] [Result] Number of tracks after filtering: 5
[2022-05-06 14:38:05,610 DEBUG two_view_estimator.py line 187 71509] Performed 2-view BA in 0.007150 seconds.


In [8]:
# Assert
rotation_angular_error = comp_utils.compute_relative_rotation_angle(
    i2Ri1_optimized, i2Ei1.rotation())
translation_angular_error = comp_utils.compute_relative_unit_translation_angle(
    i2Ui1_optimized, i2Ei1.direction()
)
assert rotation_angular_error <= 1
assert translation_angular_error <= 1
assert np.allclose(
    corr_idxs, np.hstack(
        [np.arange(normalized_coordinates_i1.shape[0]).reshape(-1, 1)] * 2)
)